# Word2Vec

## Importando-se as dependências necessárias para a atividade

In [1]:
import pandas as pd
import numpy as np
import re
import gensim
import operator
from gensim.models import Word2Vec
from unicodedata import normalize
from math import sqrt
from stop_words import get_stop_words

## Carregando o modelo no gensim

In [2]:
# Modelo com word-embeddings pré-treinados.
modelo_pre_treinado_gesim = 'pt.bin'

modelo = Word2Vec.load('pt.bin')

c:\python27\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


IOError: [Errno 2] No such file or directory: 'pt.bin.syn1neg.npy'

### Normalizando vetores

In [29]:
modelo.init_sims(replace=True)

## Obtendo-se os títulos das notícias na coleção

In [30]:
data_frame = pd.read_csv("results.csv")
titulos_colecao = data_frame["title"].tolist()

## Funções auxiliares para o processamento de texto

In [31]:
def remove_pontuacoes(texto):
    '''
        Remove-se as pontuações de um texto.
    '''
    return re.sub(r'[,.!?:*();#/&%]', '', str(texto))

def remove_caracteres_numericos(texto):
    '''
        Remove-se os caracteres numéricos de um texto
    '''
    return re.sub(r'[0-9]', '', texto)
    
def remove_acentuacoes(texto):
    '''
        Remove as acentuações de um texto.
    '''
    texto_unicode = unicode(texto, "utf-8")
    return normalize('NFKD', texto_unicode).encode('ASCII','ignore')

def converte_letras_minusculas(texto):
    '''
        Transforma todas as letras do texto em minuscúlas.
    '''
    return texto.lower()

def is_not_Vazio_ou_stop_word(token):
    '''
        Retorna um booleano indicando se o token é uma stop-word ou uma palavra vazia.
    '''
    stop_words = get_stop_words('pt')
    
    return (token not in stop_words) and token != ''
    
def remover_vazio_ou_stop_words(texto):
    '''
        Remove stop-words do texto, como: a, o, e, de, da, para, além de remover letras vazias
    '''
    lista_tokens = texto.split(" ")
    lista_token_sem_stop_words = filter(is_not_Vazio_ou_stop_word, lista_tokens)
    
    return " ".join(lista_token_sem_stop_words)

def formatar_texto_e_remover_stop_words(texto):
    '''
        Transforma o texto, eliminando acentuações, caracteres numéricos e pontuações, e
        modificando todas as letras para  minúsculas. Além disso, remove stop-words do texto.
    '''
    
    texto = remove_pontuacoes(texto)
    texto = remove_caracteres_numericos(texto)
    texto = remove_acentuacoes(texto)
    texto = converte_letras_minusculas(texto)
    texto = remover_vazio_ou_stop_words(texto)
    
    return texto

## 1) Implemente uma função que recebe uma notícia e retorna os vetores (word embeddings) das palavras do título dessa notícia a partir dos word embeddings pré-treinados com o gensim (30 pts).

In [32]:
def get_word_embeddings_noticia(noticia):
    '''
        Obtém a lista de vetores (word embeddings) para cada palavra da noticia, e retorna um dicionário,
        o qual se estrutura como: chave, a palavra da noticia, e valor, os vetores correspondentes a essa palavra.
        Obs: se a palavra não se encontra no vocabulário, então a mesma será desconsiderada do word embedding.
    '''
    dic_vetores = {}
    for palavra in noticia.split():
        if(modelo.wv.__contains__(palavra)):
            vetores = modelo.wv.__getitem__(palavra)
            dic_vetores[palavra] = vetores
        
    return dic_vetores

## 2) Implemente uma função que calcula o WMD (Word Mover's Distance) entre duas notícias usando os embeddings das palavras dos respectivos títulos  (35 pts);

In [33]:
infinito = float("inf")

def calcular_wmd_manualmente(noticia_1, noticia_2):
    '''
        Calcula-se manualmente o WMD (Word Mover's Distance) entre as duas notícias passadas por parâmetro, ou seja,
        a soma das distâncias minímas entre as palavras dessas notícias, e retorna esse valor. Para calcular, a mesma
        precisa ter posse dos embeddings das palavras de cada notícia, os quais são obtidos na questão (1). Ademais, as
        notícias precisam estar formatadas, ou seja, sem a presença de stop words, acentuações, espaços vazios, etc, para
        proporcionar resultados mais precisos.
    '''
    # Obtém-se os embeddings das palavras.
    dic_wv_noticia_1 = get_word_embeddings_noticia(noticia_1)
    dic_wv_noticia_2 = get_word_embeddings_noticia(noticia_2)
    
    wmd = 0
    for wv_noticia_1 in dic_wv_noticia_1.keys():
        
        # Define-se que até então a distância miníma entre a palavra da notícia_1 e as demais é infinito.
        min_dist = infinito
        for wv_noticia_2 in dic_wv_noticia_2.keys():
            
            # Obtém-se a menor distância entre os pontos das palavras da noticia_1 e noticia_2, considerando-se na
            # comparação, a menor obtida até o presente momento e a distância entre as palavras atuais.
            min_dist = min(min_dist, sqrt(sum((dic_wv_noticia_1[wv_noticia_1] - dic_wv_noticia_2[wv_noticia_2]) ** 2)))

        # Adiciona-se a menor distância ao wmd para ter a soma total entre as palavras das duas notícias
        wmd += min_dist * 1.0/len(dic_wv_noticia_1.keys())

    return wmd
    
    
def calcular_wmd(noticia_1, noticia_2):
    '''
        Calcula o WMD (Word Mover's Distance) entre as duas notícias passadas por parâmetro, ou seja, a soma
        das distâncias minímas entre as palavras dessas notícias, e retorna esse valor. Para calcular, a mesma
        precisa ter posse dos embeddings das palavras de cada notícia, os quais são obtidos na questão (1).
        Para realizar o cálculo, as notícias são formatadas pela função formatar_texto_e_remover_stop_words, para
        eliminar stop words, acentuações, espaços vazios, etc.
        
        OBS: Para o cálculo do wmd, utilizou-se a função wmdistance proporcionada pelo modelo do Word2Vec, não
        obstante, pode-se calcular manualmente, implementando a função como é demonstrado na função
        calcular_wmd_manualmente.
    '''
    noticia_1 = formatar_texto_e_remover_stop_words(noticia_1)
    noticia_2 = formatar_texto_e_remover_stop_words(noticia_2)

    wmd = modelo.wv.wmdistance(noticia_1,noticia_2)
    
    # Caso queira calcular manualmente, utiliza-se a função calcular_wmd_manualmente.
    # wmd = calcular_wmd_manualmente(noticia_1,noticia_2)

    return wmd

## 3) Implemente uma função que possa receber qualquer notícia como entrada e retornar as top-3 notícias mais similares (menos distantes) a ela (35 pts).

In [35]:
def get_noticias_mais_similares(titulo_noticia):
    '''
        Obtém o título das 3 notícias mais similares ao título de titulo_noticia, considerando como as mais similares
        as que possuem o menor WMD, ou seja menor distância, obtido pela função calular_wmd.
    '''
    # Tupla de dois índices, onde o primeiro índice é o título da notícia e o segundo índice é o wmd em relação ao titulo_noticia.
    lista_tupla_noticia_similaridade = []
    for titulo_noticia_comparada in titulos_colecao:
        lista_tupla_noticia_similaridade.append((titulo_noticia_comparada,
                                                 calcular_wmd(titulo_noticia, titulo_noticia_comparada)))
    
    # Ordena a lista de tuplas pelo menor wmd.
    lista_tupla_noticia_similaridade.sort(key = operator.itemgetter(1))
    # Obtém as 3 primeiras notícias, as quais são as mais similares.
    tres_mais_similares = [titulo for titulo, peso in lista_tupla_noticia_similaridade[:3]]
    
    return tres_mais_similares

## Análise

### Análise para uma consulta por "governo do Brasil"

In [54]:
resultado_governo_do_brasil = get_noticias_mais_similares("governo do Brasil")

colunas = ["Ranking", "Título"]
tabela_resultado_governo_do_brasil = pd.DataFrame({"Título": resultado_governo_do_brasil,
                                 "Ranking": xrange(1, 4)})
tabela_resultado_governo_do_brasil = tabela_resultado_governo_do_brasil.reindex(columns=colunas)

tabela_resultado_governo_do_brasil

,Ranking,Título
0,1,Bolsonaro (des)governa o Brasil pelo Twitter
1,2,Governo Bolsonaro prega “negacionismo históric...
2,3,Instruído por ministros Bolsonaro dá sinal ve...


Observe na tabela acima que todos títulos estão relacionados ao governo Bolsonaro que por conseguinte o atual governo do Brasil, os quais, apresentam pontos do governo. Logo, para essa pesquisa, o algoritmo funcionou como o esperado, retornando títulos similares ao esperado.

### Análise para uma consulta por "tortura na ditadura"

In [49]:
resultado_tortura_ditadura = get_noticias_mais_similares("tortura na ditadura")

colunas = ["Ranking", "Título"]
tabela_resultado_tortura_ditadura = pd.DataFrame({"Título": resultado_tortura_ditadura,
                                 "Ranking": xrange(1, 4)})
tabela_resultado_tortura_ditadura = tabela_resultado_tortura_ditadura.reindex(columns=colunas)

tabela_resultado_tortura_ditadura

,Ranking,Título
0,1,“Lógica de usar torturadores da ditadura no cr...
1,2,Na Argentina falar da ditadura e dos militare...
2,3,Cúpula militar uruguaia cai após revelação de ...


Observe na tabela acima, são apresentados títulos referentes à ditadura e à tortura, como era o esperado. Isso novamente constata que o algoritmo está funcionando.

### Análise pela consulta "Bolsonaro troca embaixada por escritório em Jerusalém  mas não evita retaliação palestina", o qual é um título presente no conjunto de dados.

In [53]:
resultado_titulo_conjunto_dados = get_noticias_mais_similares(
    "Bolsonaro troca embaixada por escritório em Jerusalém  mas não evita retaliação palestina")

colunas = ["Ranking", "Título"]
tabela_resultado_titulo_conjunto_dados = pd.DataFrame({"Título": resultado_titulo_conjunto_dados,
                                 "Ranking": xrange(1, 4)})
tabela_resultado_titulo_conjunto_dados = tabela_resultado_titulo_conjunto_dados.reindex(columns=colunas)

tabela_resultado_titulo_conjunto_dados

,Ranking,Título
0,1,Bolsonaro troca embaixada por escritório em Je...
1,2,Após acenar com embaixada Bolsonaro anuncia e...
2,3,Rota mata 11 pessoas em Guararema na terceira...


Observe que o título considerado mais similar é o próprio título passado, além desse, o segundo também fala sobre a embaixada nem Jerusalém, enquanto que o terceiro pouco faz similaridade. Não obstante, ainda assim, o cálculo se apresenta funcional para buscar pela similaridade.